In [1]:
import pandas as pd

In [3]:
pd.read_hdf('../data/gss_eda.hdf5')

,year,id_,agewed,divorce,sibs,childs,age,educ,paeduc,maeduc,...,fejobaff,discaffm,discaffw,fehire,meovrwrk,avoidbuy,income,rincome,realrinc,china
0,1972,1,NaN,NaN,3.0,0.0,23.0,16.0,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-1
1,1972,2,21.0,2.0,4.0,5.0,70.0,10.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-1
2,1972,3,20.0,2.0,5.0,4.0,48.0,12.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-1
3,1972,4,24.0,2.0,5.0,0.0,27.0,17.0,16.0,12.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-1
4,1972,5,22.0,2.0,2.0,2.0,61.0,12.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64809,2018,2344,NaN,NaN,9.0,2.0,37.0,12.0,12.0,13.0,...,1.0,2.0,NaN,NaN,4.0,0,NaN,NaN,NaN,-1
64810,2018,2345,NaN,2.0,9.0,4.0,75.0,12.0,8.0,8.0,...,1.0,2.0,NaN,NaN,4.0,0,12.0,10.0,7377.5,-1
64811,2018,2346,NaN,1.0,3.0,3.0,67.0,12.0,12.0,14.0,...,NaN,NaN,NaN,NaN,NaN,0,12.0,3.0,1589.0,-1
64812,2018,2347,NaN,1.0,1.0,2.0,72.0,16.0,14.0,12.0,...,3.0,2.0,NaN,NaN,2.0,0,12.0,NaN,NaN,-1


In [4]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/BiteSizeBayes/raw/master/gss_bayes.csv')

Downloaded gss_bayes.csv


In [5]:
gss_df = pd.read_csv('gss_bayes.csv')
gss_df.head(3)

,caseid,year,age,sex,polviews,partyid,indus10
0,1,1974,21.0,1,4.0,2.0,4970.0
1,2,1974,41.0,1,5.0,0.0,9160.0
2,5,1974,58.0,2,6.0,1.0,2670.0


In [7]:
gss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49290 entries, 0 to 49289
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   caseid    49290 non-null  int64  
 1   year      49290 non-null  int64  
 2   age       49290 non-null  float64
 3   sex       49290 non-null  int64  
 4   polviews  49290 non-null  float64
 5   partyid   49290 non-null  float64
 6   indus10   49290 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 2.6 MB


### polviews

1   Extremely liberal
2   Liberal
3   Slightly liberal
4   Moderate
5   Slightly conservative
6   Conservative
7   Extremely conservative

### partyid

0   Strong democrat
1   Not strong democrat
2   Independent, near democrat
3   Independent
4   Independent, near republican
5   Not strong republican
6   Strong republican
7   Other party


### banker = indus10 @6870


### What's the probability that a respondent is a Democrat, given that they are liberal?

In [10]:
def calc_prob(data):
    return data.mean()

democrat = gss_df['partyid'] <= 1
liberal = gss_df['polviews'] <= 3

calc_prob(democrat | liberal)

0.49748427672955975

In [12]:
calc_prob(democrat[liberal])

0.5206403320240125

What is the probability that a respondent is female, given that they are a banker?

In [13]:
female = gss_df['sex'] == 2
calc_prob(female)

0.5378575776019476

In [14]:
banker = gss_df['indus10'] == 6870

calc_prob(female[banker])

0.7706043956043956

In [16]:
def conditional(proposition, given):
    return calc_prob(proposition[given])

In [17]:
conditional(liberal, given=female)

0.27581004111500884

### important! conditional probability is not commutative

calc_prob(A & B) != calc_prob(B & A)

In [19]:
# how many bankers are female?
calc_prob(female[banker])

0.7706043956043956

In [21]:
# how many bankers are females?
calc_prob(banker[female])

# Only about 2% of female respondents are bankers.

0.02116102749801969

In [22]:
conditional(female, given=liberal & democrat)

0.576085409252669

Linda is 31 years old, single, outspoken, and very bright. She majored in philosophy. As a student, she was deeply concerned with issues of discrimination and social justice, and also participated in anti-nuclear demonstrations. Which is more probable?

1. Linda is a banker.

2. Linda is a banker and considers herself a liberal Democrat.


To answer this question, compute

- The probability that Linda is a female banker,

- The probability that Linda is a liberal female banker, and

- The probability that Linda is a liberal female banker and a Democrat.

In [23]:
conditional(female, given=banker)

0.7706043956043956

In [24]:
conditional(female & liberal, given=banker)

0.17307692307692307

In [26]:
conditional(female & liberal & democrat, given=banker)

0.08379120879120878

If you are not a liberal at 25, you have no heart. If you are not a conservative at 35, you have no brain.

- What is the probability that a randomly chosen respondent is a young liberal?

- What is the probability that a young person is liberal?

- What fraction of respondents are old conservatives?

- What fraction of conservatives are old?

In [28]:
young = gss_df['age'] < 30
old = gss_df['age'] >= 65

calc_prob(young[liberal])

0.24034684651300675